NOTES - 
I used pandas and did exact matching

In [1]:
import csv
import pandas as pd
import numpy as np

# SPARK DataFrame

Fuzzymatch on restaurant name

In [27]:
import pyspark
sc = pyspark.SparkContext(appName="myAppName")

In [28]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [29]:
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [31]:
from pyspark.sql import functions as F
from pyspark.sql.functions import upper, col, regexp_extract, regexp_replace
from pyspark.sql.functions import levenshtein 

In [32]:
#dt = ''DataSources/all_clean_restaurants.csv''
biz = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load('DataSources/all_clean_restaurants.csv')
mybiz = biz


In [33]:
green = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load('DataSources/clean_green.csv')
mygreen = green


In [34]:
mybiz.show()


Row(_c0=1, business_id='QXAEGFB4oINsVuTFxEYKFQ', categories='SPECIALTY FOOD, RESTAURANTS, DIM SUM, IMPORTED FOOD, FOOD, CHINESE, ETHNIC FOOD, SEAFOOD', name='EMERALD CHINESE RESTAURANT', review_count=128, stars=2.5, address='30 EGLINTON AVENUE W', city='MISSISSAUGA', state='ON', postal_code='L5R 3E7', latitude=43.605498974300005, longitude=-79.65228890899999)

In [49]:
mygreen = mygreen.withColumnRenamed('Name', 'green_name')
mygreen = mygreen.drop('Address', 'City', 'State', 'Zip')

In [50]:
mygreen.show()

+---+--------------------+------+
|_c0|          green_name|rating|
+---+--------------------+------+
|  0|CROWN CAFE AT STA...|     4|
|  1|              BOLOCO|     2|
|  5|             VESELKA|     2|
|  6|       GRENDELS DEN|     2|
|  8|      BOXCAR CANTINA|     2|
|  9|INTERNATIONAL VIL...|     3|
| 10|SIGNAL MOUNTAIN L...|     3|
| 11|       WINSOR SCHOOL|     2|
| 12|   THE DALTON SCHOOL|     3|
| 13|  AROMA THYME BISTRO|     3|
| 14|  HORACE MANN SCHOOL|     2|
| 15|  OCCASIONS CATERERS|     3|
| 16|    FOUNDING FARMERS|     3|
| 17|          CAFE MANNA|     3|
| 18|GRAND CANYON LODG...|     2|
| 19|            PISTICCI|     4|
| 20|EDGEWOOD COLLEGE ...|     3|
| 21|THE GREENHOUSE TA...|     3|
| 22|BAMBOO SUSHI SOUT...|     3|
| 23|LOU'S RESTAURANT ...|     2|
+---+--------------------+------+
only showing top 20 rows



In [52]:
joinedDF = mybiz.join(mygreen, levenshtein(mybiz["name"], mygreen["green_name"]) < 3) 

In [53]:
joinedDF = joinedDF.drop('_c0')

In [54]:
joinedDF.head()

Row(business_id='9E1q2uEMd881wnruicNTUA', categories='SPORTS BARS, NIGHTLIFE, CHICKEN WINGS, RESTAURANTS, BARS, AMERICAN (TRADITIONAL)', name='BUFFALO WILD WINGS', review_count=133, stars=3.0, address='8870 E INDIAN BEND RD', city='SCOTTSDALE', state='AZ', postal_code='85250', latitude=33.5385580506, longitude=-111.8908726255, green_name='BUFFALO WILD WINGS', rating=1)

In [55]:
joinedDF.write.csv("business_and_green.csv")

____________________________________________________________________________________

# PANDAS

exact match on name

In [4]:
biz = pd.read_csv('DataSources/all_clean_restaurants.csv', header=0, delimiter=',')

In [5]:
green = pd.read_csv('DataSources/clean_green.csv', header=0, delimiter=',')

In [ ]:
biz = biz.sort_values(['name'])
green = green.sort_values(['Name'])

In [21]:
biz.head()

,Unnamed: 0,business_id,categories,name,review_count,stars,address,city,state,postal_code,latitude,longitude
40090,99939,ysaGG0hm7-ug-IuRd-8_ew,"HAWAIIAN, CHINESE, RESTAURANTS, JAPANESE",#1 FRIED RICE,44,3.5,9310 W VAN BUREN ST,TOLLESON,AZ,85353,33.450581,-112.260313
42477,105935,42yLya7lhe7TQGz3KEdKAw,"RESTAURANTS, HAWAIIAN, SEAFOOD, BARBEQUE",#1 HAWAIIAN BARBECUE,40,3.5,"5905 S EASTERN AVE, STE 105",LAS VEGAS,NV,89119,36.081920,-115.119526
15547,39188,gA78_OGp1ekMvZWjRoBmFw,"RESTAURANTS, HAWAIIAN, BARBEQUE, SEAFOOD",#1 HAWAIIAN BARBECUE,23,4.0,"7960 S RAINBOW BLVD, STE 8000G",LAS VEGAS,NV,89139,36.043305,-115.243062
18021,45372,AhD9x2NNFqPvVwmyEZU5Cg,"RESTAURANTS, VIETNAMESE",#1 PHO,8,3.5,7778 W 130TH ST,MIDDLEBURG HEIGHTS,OH,44130,41.351727,-81.785527
44476,110881,kkEqZmVvVkgmCaOqE13mDg,"ASIAN FUSION, RESTAURANTS, JAPANESE, SUSHI BARS",#1 SUSHI,64,4.0,9617 N METRO PKWY,PHOENIX,AZ,85051,33.575127,-112.120880


In [22]:
green.head()

,Unnamed: 0,Name,Address,City,State,Zip,rating
296,322,&GRAIN,700 NORTH AVENUE,GARWOOD,NJ,7027,3
540,570,19515-THE 459 COMMONS,THE 459 COMMONS,BATON ROUGE,LA,70803,3
544,574,1ST AVENUE COFFEE,1011 WEST 1ST AVENUE,SPOKANE,WA,99201,1
398,427,24/7 DINING HALL AT UNIVERSITY OF OTTAWA,85 UNIVERSITY (345),OTTAWA,ONTARIO,K1N 6N5,3
563,593,50 BURGER,445 EAST HIGHWAY 50,SALIDA,CO,81201,3


In [24]:
green.isnull().sum()

Unnamed: 0     0
Name           0
Address       32
City          32
State         34
Zip           51
rating         0
dtype: int64

In [17]:
testmerge = biz.merge(green, left_on='name', right_on='Name', how='inner', suffixes=['_b', '_g'])



In [56]:
testmerge

,Unnamed: 0_b,business_id,categories,name,review_count,stars,address,city,state,postal_code,latitude,longitude,Unnamed: 0_g,Name,Address,City,State,Zip,rating
0,114860,5ccc6zVHPggiA33df7V1NQ,"SUSHI BARS, JAPANESE, RESTAURANTS",BENTO SUSHI,4,3.0,150 KING STREET W,TORONTO,ON,M5H 1J9,43.648116,-79.384479,495,BENTO SUSHI,NaN,NaN,NaN,NaN,1
1,133936,cFvmSY2XBy10KGXF7l612A,"RESTAURANTS, SUSHI BARS, JAPANESE",BENTO SUSHI,6,3.0,4700 130 AVENUE SE,CALGARY,AB,T2Z 4E7,50.934255,-113.967170,495,BENTO SUSHI,NaN,NaN,NaN,NaN,1
2,75096,_JMzP19EadnMXn_E5tRFjg,"SUSHI BARS, JAPANESE, RESTAURANTS",BENTO SUSHI,52,3.0,7700 HURONTARIO STREET,BRAMPTON,ON,L6Y 4M3,43.658735,-79.727002,495,BENTO SUSHI,NaN,NaN,NaN,NaN,1
3,59193,OqP2SGjXgmKTpdRehjyCVw,"SUSHI BARS, RESTAURANTS",BENTO SUSHI,4,1.0,751 3 ST SW,CALGARY,AB,T2P 4K8,51.046114,-114.070937,495,BENTO SUSHI,NaN,NaN,NaN,NaN,1
4,51963,qX3YZQZHXeiFbcIc1DZzgA,"SUSHI BARS, RESTAURANTS",BENTO SUSHI,13,2.0,"TERMINAL 1, LEVEL 2, TORONTO PEARSON INTERNATI...",MISSISSAUGA,ON,L4V 1W1,43.682575,-79.615966,495,BENTO SUSHI,NaN,NaN,NaN,NaN,1
5,26159,54pbmQvIJDYHkRHuIWCsqA,"NIGHTLIFE, BARS, BEER GARDENS",BLACKBIRD,3,4.5,119 W MAIN ST,URBANA,IL,61801,40.112410,-88.207802,49,BLACKBIRD,619 W RANDOLPH,CHICAGO,IL,60661,2
6,175527,BDTIaZHqgBbV1aqlh1l_Vg,"FOOD, BARS, PUBS, AMERICAN (TRADITIONAL), REST...",BUFFALO WILD WINGS,57,2.0,"2109 W ROSSEVELT BLVD, SUITE F",MONROE,NC,28110,35.004693,-80.557187,497,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
7,116706,o3SafLqbLT_zT2HyqHwvhg,"CHICKEN WINGS, BARS, AMERICAN (TRADITIONAL), S...",BUFFALO WILD WINGS,32,2.0,9062 STATE ROUTE 14,STREETSBORO,OH,44241,41.237812,-81.336614,497,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
8,91452,kRlzBlhoD9P__yElGJGTcg,"CHICKEN WINGS, SPORTS BARS, NIGHTLIFE, BARS, R...",BUFFALO WILD WINGS,35,3.0,32914 WALKER ROAD,AVON LAKE,OH,44012,41.493672,-82.022615,497,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
9,155790,eBoRFMm14n6Bte5vJIaeSg,"RESTAURANTS, SPORTS BARS, CHICKEN WINGS, BARS,...",BUFFALO WILD WINGS,56,2.0,12459 CEDAR RD,CLEVELAND,OH,44106,41.501307,-81.592228,497,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1


In [20]:
##26007
biz.duplicated(subset='name').sum()

26007

In [58]:
testmerge = testmerge.drop(['Unnamed: 0_g','Unnamed: 0_b'], axis=1)

In [59]:
testmerge


,business_id,categories,name,review_count,stars,address,city,state,postal_code,latitude,longitude,Name,Address,City,State,Zip,rating
0,5ccc6zVHPggiA33df7V1NQ,"SUSHI BARS, JAPANESE, RESTAURANTS",BENTO SUSHI,4,3.0,150 KING STREET W,TORONTO,ON,M5H 1J9,43.648116,-79.384479,BENTO SUSHI,NaN,NaN,NaN,NaN,1
1,cFvmSY2XBy10KGXF7l612A,"RESTAURANTS, SUSHI BARS, JAPANESE",BENTO SUSHI,6,3.0,4700 130 AVENUE SE,CALGARY,AB,T2Z 4E7,50.934255,-113.967170,BENTO SUSHI,NaN,NaN,NaN,NaN,1
2,_JMzP19EadnMXn_E5tRFjg,"SUSHI BARS, JAPANESE, RESTAURANTS",BENTO SUSHI,52,3.0,7700 HURONTARIO STREET,BRAMPTON,ON,L6Y 4M3,43.658735,-79.727002,BENTO SUSHI,NaN,NaN,NaN,NaN,1
3,OqP2SGjXgmKTpdRehjyCVw,"SUSHI BARS, RESTAURANTS",BENTO SUSHI,4,1.0,751 3 ST SW,CALGARY,AB,T2P 4K8,51.046114,-114.070937,BENTO SUSHI,NaN,NaN,NaN,NaN,1
4,qX3YZQZHXeiFbcIc1DZzgA,"SUSHI BARS, RESTAURANTS",BENTO SUSHI,13,2.0,"TERMINAL 1, LEVEL 2, TORONTO PEARSON INTERNATI...",MISSISSAUGA,ON,L4V 1W1,43.682575,-79.615966,BENTO SUSHI,NaN,NaN,NaN,NaN,1
5,54pbmQvIJDYHkRHuIWCsqA,"NIGHTLIFE, BARS, BEER GARDENS",BLACKBIRD,3,4.5,119 W MAIN ST,URBANA,IL,61801,40.112410,-88.207802,BLACKBIRD,619 W RANDOLPH,CHICAGO,IL,60661,2
6,BDTIaZHqgBbV1aqlh1l_Vg,"FOOD, BARS, PUBS, AMERICAN (TRADITIONAL), REST...",BUFFALO WILD WINGS,57,2.0,"2109 W ROSSEVELT BLVD, SUITE F",MONROE,NC,28110,35.004693,-80.557187,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
7,o3SafLqbLT_zT2HyqHwvhg,"CHICKEN WINGS, BARS, AMERICAN (TRADITIONAL), S...",BUFFALO WILD WINGS,32,2.0,9062 STATE ROUTE 14,STREETSBORO,OH,44241,41.237812,-81.336614,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
8,kRlzBlhoD9P__yElGJGTcg,"CHICKEN WINGS, SPORTS BARS, NIGHTLIFE, BARS, R...",BUFFALO WILD WINGS,35,3.0,32914 WALKER ROAD,AVON LAKE,OH,44012,41.493672,-82.022615,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1
9,eBoRFMm14n6Bte5vJIaeSg,"RESTAURANTS, SPORTS BARS, CHICKEN WINGS, BARS,...",BUFFALO WILD WINGS,56,2.0,12459 CEDAR RD,CLEVELAND,OH,44106,41.501307,-81.592228,BUFFALO WILD WINGS,NaN,NaN,NaN,NaN,1


In [60]:
testmerge.to_csv('business_green.csv')